
## ***Multilayer Perceptrons from Scratch*** ##



#### ***1. Motivation*** ####

Using a linear regression model as a starting point, a non-linear neural network can be implemented. Non-linear models can identify more complex patterns than linear models, which makes these neural networks more useful for regression tasks. By introducing perceptrons, which require an activation function, one can implement a multilayer neural network from scratch that can be used to solve complex regression problems.


#### ***2. Introducing Perceptrons*** ####

##### *2.1 Extension of Linear Regression* #####

A linear regression model can be extended into a non-linear model by introducing hidden layers with *perceptrons* as the nodes. Between the input and output layers, there can be any number of interconnected hidden layers in a neural network. In a *feedfoward neural network*, the nodes are connected without loops, which makes for a relatively simple model to implement from scratch. Data flow and are transformed from one layer to another by computations that include multiplying by the model's weights and adding a bias term. An *activation function* is applied to nodes in each hidden layer to make the computations non-linear. 

The Model class below is used to implement the neural network. The layers array includes the number of nodes in each layer of the neural network, including the input, hidden, and output layers. Random weights and biases are initialized in their own arrays of matrices that connect neighboring layers. 

In [63]:
import numpy as np
import utilities as util

class Model:
    
    def __init__(self, layers):
        
        self.layers = layers
        self.weights = []
        self.biases = []

        # initialize random weights and biases
        rng = np.random.default_rng()
        for i in range(len(layers) - 1):
            self.weights.append(rng.random((layers[i], layers[i+1])))
            self.biases.append(rng.random((1, layers[i+1])))


##### *2.2 Activation Functions: The Non-Linearity of Perceptrons* #####

Out of the many activation functions that can be applied to perceptrons, the activation function of choice for this neural network is the simple but effective ReLU, which stands for Rectified Linear Unit. Interestingly, as the name suggests, the plot of the function roughly resembles the current-voltage curve of non-linear devices, or rectifiers. ReLU is piecewise linear, and when the input is less than zero, the output is zero, otherwise the output equals the input. This activation function is especially great to work with due to its derivative, which is either zero or one and is often defined to be zero at the breakpoint. 

The activation function that will be applied to the hidden layers is implemented below.  

In [58]:
def relu(A):
    zeros = np.zeros(A.shape)
    return np.maximum(A, zeros)


#### ***3. Training the Neural Network*** ####

##### *3.1 Predicting the Outputs* #####

Forward propagation between layers is an extension of linear regression, which includes multiplying data by weights and adding a bias, followed by activation. In some cases, an activation function may be applied to the output layer to constrain the outputs, but not always. In this neural network, the outputs can be any numerical values, and ReLU is applied only to the hidden layers. The method below is added to the Model class.

In [120]:
@util.add_to_class(Model)
def predict(self, input, save_activated=None):
    
        last_predicted = input.copy()

        for i in range(len(self.layers) - 1):
            b_matrix = np.tile((self.biases[i]), (len(last_predicted), 1))
            last_predicted = np.array(np.matmul(last_predicted, self.weights[i]) + b_matrix)
            
            if i is not (len(self.layers) - 2):
                last_predicted = relu(last_predicted)
                
            if save_activated is not None:
                save_activated.append(last_predicted)

        return last_predicted


It should be noted that gradient descent will be used to update the model's weights in a backpropagation process. Thus, as they are computed, the activated layers are saved in an array for later use. The last predicted layer contains the predicted outputs, $\hat{y}$, which will be used to calculate the error.


##### *3.2 The Objective Function: Introducing Regularization with the Frobenius Norm* #####

Similarly to the linear regression example, the loss function, $L$, for this model is MSE. However, in order to prevent overfitting, a *regularization term* 

$$ s = \frac{\lambda}{2} \sum_{i} \lVert \theta_i \rVert_F^2 $$ 

is added, where $\lambda$ is a weight decay parameter, usually between $0.0$ and $0.1$, and the sum of the squared Frobenius norms of the weight matrices is taken. 

In [56]:
def frob_squared(A):
    return np.trace(np.matmul(np.transpose(A), A))

In this neural network, the *objective function*

$$ J = L + s $$

is the function to be minimized through gradient descent. 


##### *3.3 Backpropagating the Gradient* #####

Gradient descent - the same algorithm used for optimization of coefficients in linear regression - is used to update the weights. 
Due to hidden layers which undergo activation, the computation of the gradient has more steps than previously discussed for linear regression. The gradient $ \partial J/\partial \theta_i $ for each weight matrix $\theta_i$ can be computed by applying chain rule to the hidden layer outputs and the derivative of the activation function. A brief walk-through of essential derivations is provided below. 

Both $\partial J/\partial L$ and $\partial J/\partial s$ are $1$. Further computations of $\partial L/\partial \theta_i$ and $\partial s/\partial \theta_i$ need to be made. 

Starting at the output layer and substituting $\hat{y} = g(z^{(f)})$, where $g$ is the activation function and $z^{(f)}$ contains the values from the last hidden layer, the gradiet due to loss can be computed as

$$ \frac{\partial L}{\partial \theta_i} = \frac{1}{N} \sum_{j=1}^N (y_j - g(z_{j}^{(f)})) \cdot \frac{\partial g(z^{(f)})}{\partial \theta_i} $$

When computing the derivatives using chain rule, the derivative of the activation function, such as ReLU, is multiplied by the derivative of the layer being activated. This expands the computation to 

$$ \frac{\partial L}{\partial \theta_i} =  \frac{1}{N} \sum_{j=1}^N (y_j - g(z_{j}^{(f)})) \cdot g^\prime(a_{j}^{(i-1)} \theta_i + b_i) \cdot a^{(i-1)} $$

The entire summation term can be called $\Delta \theta_i$ for later reference. When computing the gradients for earlier layers, the same approach of applying chain rule can be taken. 

The gradient from the regularization term can be computed as


$$ \frac{\partial s}{\partial \theta_i} = \lambda \theta_i $$

Combining the gradients due to loss and regularization results in the following updates to the weights and biases:

$$ \theta_i = \theta_i - \alpha [ (\frac{1}{N} \Delta \theta_i) + \lambda \theta_i]$$

$$ b_i = b_i - \alpha [\frac{1}{N} \Delta b_i] $$


where $\alpha$ is the learning rate and $N$ is the number of samples training the neural network. 

##### *3.4 Putting It All Together* #####

Training consists of predicting the outputs, calculating the errors, and updating the weights and biases of the model. The function below implements training the model. 

In [217]:
@util.add_to_class(Model)
def train(self, inputs, outputs, num_epochs=5, lr=0.01, t=1000):
    num_samples = len(inputs)
    l = 1e-5 # weight decay parameter
    
    for epoch in range(num_epochs):
        for run in range(t):
            # reset the activated layers and sum of squared Frobenius norms of weights matrices
            s = 0
            activated = []

            # predict the outputs
            y_hat = self.predict(inputs, activated)
    
            # compute the errors and objective
            errors = y_hat - outputs
            loss = 1/2*(np.dot(errors[:,0], errors[:,0])**2)
    
            for theta in self.weights:
                s += frob_squared(theta)
    
            objective = 1/num_samples*loss + l/2*s
    
            # log the objective every 100 runs
            if run % 100 == 99:
                print(f"{run}: {objective}")
    
            # compute z primes
            deltas = self.weights.copy()
            z_primes = activated.copy()
            z_primes[len(activated) - 1] = errors
    
            for j in range(len(self.layers) - 2):
                a_primes = np.array(activated[len(self.layers) - 3 - j].copy())
                a_primes[a_primes > 0] = 1.0
                prev_z_prime = (np.multiply(
                                    np.matmul(z_primes[len(activated)-1-j], np.transpose(self.weights[len(self.layers)-2-j])), 
                                    a_primes))
                z_primes[len(activated) - 2 - j] = prev_z_prime
    
            # compute deltas
            deltas[0] = (np.matmul(np.transpose(inputs), z_primes[0]))
            for j in range(len(self.layers) - 3):
                deltas[j+1] = (np.matmul(np.transpose(activated[j]), z_primes[j+1]))
    
            # update weights and biases
            for j in range(len(self.weights) - 1):
                self.weights[j] = self.weights[j] - lr*(1/num_samples*deltas[j] + l*self.weights[j])
                self.biases[j] = self.biases[j] - lr/num_samples*np.sum(z_primes[j], axis=0)
                
        print(f"\nEpoch {epoch+1}: {y_hat}\n")
    
    return y_hat


##### 3.5 Neural Net Example #####


In [213]:
layers = (2, 3, 5, 2, 1)
data = np.array([[1, 0.5], [1, 1], [3, 3], [0.5, 1], [3, 2], [2.5, 1.5]])
outcomes = np.array([[1.25], [2], [12], [1.5], [7], [4.75]])   # first + second^2

neural_net = Model(layers)
predicted = neural_net.train(data, outcomes)


99: 6.085642574048144
199: 0.33351983259576096
299: 0.02866613721719484
399: 0.003720760844375917
499: 0.0012717620825654645
599: 0.0008248930180855524
699: 0.0007021940144321414
799: 0.000645732883272914
899: 0.0006172330887271987
999: 0.0006028268417983063

Epoch 1: [[ 1.23982529]
 [ 1.87909969]
 [11.94381113]
 [ 1.62871017]
 [ 7.17593281]
 [ 4.62564826]]

99: 0.0005896939678572123
199: 0.0005770686259332289
299: 0.0005647949749726724
399: 0.0005527917163813295
499: 0.0005410511722633253
599: 0.0005295256610297155
699: 0.0005182590404130314
799: 0.000507209099022288
899: 0.000496411012342344
999: 0.0004858682510294627

Epoch 2: [[ 1.24478915]
 [ 1.89668041]
 [11.94733614]
 [ 1.61272036]
 [ 7.17220554]
 [ 4.62427663]]

99: 0.0004754911299646224
199: 0.0004653684304947161
299: 0.00045548075313369134
399: 0.00044580734991361604
499: 0.0004363504369141104
599: 0.0004271139905374439
699: 0.0004142914811181046
799: 0.00040091688229794653
899: 0.0003877727691739443
999: 0.000374560957434437